In [ ]:
# Import our dependencies:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import pandas as pd
import tensorflow as tf

In [ ]:
# Import our input dataset:
attrition_df = pd.read_csv("/Users/diegoperezo97/Documents/Tecnológico de Monterrey – Data Analytics Boot Camp/neural-network-charity-analysis/Module 19/data/HR-Employee-Attrition.csv")
attrition_df.head()

In [ ]:
# Generate our categorical variable list:
attrition_cat = attrition_df.dtypes[attrition_df.dtypes == "object"].index.tolist()

In [ ]:
# Check the number of unique values in each column:
attrition_df[attrition_cat].nunique()

In [ ]:
# Create a OneHotEncoder instance:
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list:
encode_df = pd.DataFrame(enc.fit_transform(attrition_df[attrition_cat]))

# Add the encoded variable names to the DataFrame:
encode_df.columns = enc.get_feature_names(attrition_cat)
encode_df.head()

In [ ]:
# Marge one-hot encoded features and drop the originals:
attrition_df = attrition_df.merge(encode_df, left_index=True, right_index=True)
attrition_df = attrition_df.drop(attrition_cat, 1)
attrition_df

In [ ]:
# Split our preprocessed data into our features and target arrays:
y = attrition_df["Attrition_Yes"].values
X = attrition_df.drop(["Attrition_Yes", "Attrition_No"], 1).values

# Split the preprocessed data into training and testing dataset:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [ ]:
# Crate a StandardScaler instance:
scaler = StandardScaler()

# Fit the StandardScaler:
X_scaler = scaler.fit(X_train)

# Scale the data:
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
# Define the model - deep neural net:
number_input_features = len(X_train[0])
hidden_nodes_layers1 = 8
hidden_nodes_layers2 = 5

nn = tf.keras.models.Sequential()

# First hidden layer:
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layers1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer:
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layers2, activation="relu")
)

# Output layer:
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model:
nn.summary()

In [ ]:
# Import checkpoint dependencies:
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames:
os.makedirs("checkpoints/", exist_ok="True")
checkpoint_path = f"/Users/diegoperezo97/Documents/Tecnológico de Monterrey – Data Analytics Boot Camp/neural-network-charity-analysis/Module 19/checkpoints/weights.{epoch:02d}.hdf5"

In [ ]:
# Compile the model:
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
# Create a callback that saves the model's weights every epoch:
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq="epoch"
)

In [ ]:
# Train the model:
fit_model = nn.fit(X_train, y_train, epochs=100)

In [ ]:
# Evaluate the model using the test data:
model_loss, model_accuracy = nn.evaluate(X_test, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
# Define the model - deep neural net:
number_input_features = len(X_train_scaled[0])
hidden_nodes_layers1 = 8
hidden_nodes_layers2 = 5

nn_new = tf.keras.models.Sequential()

# First hidden layer:
nn_new.add(
    tf.keras.layers.Dense(units=hidden_nodes_layers1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer:
nn_new.add(
    tf.keras.layers.Dense(units=hidden_nodes_layers2, activation="relu")
)

# Output layer:
nn_new.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model:
nn_new.summary()

# Compile the model:
nn_new.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Restore the model weights:
nn_new.load_weights("/Users/diegoperezo97/Documents/Tecnológico de Monterrey – Data Analytics Boot Camp/neural-network-charity-analysis/Module 19/checkpoints/weights.100.hdf5")

# Evaluate the model using the test data:
model_loss, model_accuracy = nn_new.evaluate(X_test, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
# Export our model to HDF5 file:
nn_new.save("trained_attrition.h5")

In [ ]:
# Import the model to a new object:
nn_imported = tf.keras.models.load_model("trained_attrition.h5")

In [ ]:
# Evaluate the model using the test data:
model_loss, model_accuracy = nn_imported.evaluate(X_test_scaled, y_test, verbose=2)